In [ ]:
###访问Amazon.com###
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#关闭chrome弹窗通知，页面化时使用
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
chrome_options = Options()
#无可视化界面禁用
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--no-sandbox') #解决DevToolsActivePort文件不存在的报错
chrome_options.add_argument("disable-infobars") # 关闭'chrome正受到自动测试软件的控制'提示
chrome_options.add_argument('window-size=1920x3000')  #指定浏览器分辨率
chrome_options.add_argument('--disable-gpu')  #谷歌文档提到需要加上这个属性来规避bug
chrome_options.add_argument('--hide-scrollbars')  #隐藏滚动条, 应对一些特殊页面
#chrome_options.add_argument('blink-settings=imagesEnabled=false')  #不加载图片, 提升速度

#chrome_options.add_argument('--headless')  #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
chrome_options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" #手动指定本机电脑使用的浏览器位置

# 创建driver,进行后面的请求页面等操作，executable_path指定本机中chromedriver.exe的位置
driver = webdriver.Chrome(options=chrome_options, executable_path=r"C:\Users\UNclePeach\AnacondaProjects\chromedriver.exe") 

In [ ]:
#从link.csv文件中逐行读取(仅处理SKU,Amazon link齐备的行,剔除header行)#
import os
import csv
import requests
import PIL.Image as Image
from io import BytesIO
import traceback

IMAGES_FORMAT = ['.jpg', '.JPG', '.png', '.PNG']  # 图片格式
IMAGE_SIZE = 366  # 每张小图片的大小，可自行调节大小
IMAGE_COLUMN = 1  # 图片间隔，也就是合并成一张图后，一共有几列

# 定义图像拼接函数
def image_compose():
    to_image = Image.new('RGB', (IMAGE_COLUMN * IMAGE_SIZE, IMAGE_ROW * IMAGE_SIZE)) 
     # 创建一个新图
    # 循环遍历，把每张图片按顺序粘贴到对应位置上
    for y in range(1, IMAGE_ROW + 1):
        for x in range(1, IMAGE_COLUMN + 1):
            from_image = Image.open(IMAGES_PATH + image_names[IMAGE_COLUMN * (y - 1) + x - 1]).resize(
                (IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
            to_image.paste(from_image, ((x - 1) * IMAGE_SIZE, (y - 1) * IMAGE_SIZE))
    return to_image.save(IMAGE_SAVE_PATH)  # 保存新图

with open('links.csv','rt',encoding = "ISO-8859-1") as f: 
    cr = csv.reader(f)
    for row in cr:
        if row[1] is not "" and row[6] is not "" and row[1] != "SKU / UPC" and "/dp/" in row[6]:
            i = 0
            SKU = row[1]
            link = row[6]
            driver.get(link)
            buttons = driver.find_elements_by_xpath("//div[@id='altImages']//ul/li/span/span/span/span/img")
            for button in buttons:
                ActionChains(driver).move_to_element(button).perform()
            print(SKU,"end of active")
            imgs = driver.find_elements_by_xpath("//div[@id='main-image-container']//span/div/img")

            IMAGES_PATH = "C:/Users/UNclePeach/Downloads/IMG/"+ SKU +"/"             
            if not os.path.exists(IMAGES_PATH):
                os.makedirs(IMAGES_PATH)
                
            for img in imgs:
                url = img.get_attribute("data-old-hires")
                try:
                    response = requests.get(url)
                    image = Image.open(BytesIO(response.content))
                    i = i + 1
                    image.save(IMAGES_PATH+str(i)+".jpg")
                except:
                    continue
                    
            IMAGE_ROW = i
            IMAGE_SAVE_PATH = IMAGES_PATH + "final.jpg"
            image_names = [name for name in os.listdir(IMAGES_PATH) for item in IMAGES_FORMAT if os.path.splitext(name)[1] == item]
            if len(image_names) != IMAGE_ROW * IMAGE_COLUMN:
                        raise ValueError("合成图片的参数和要求的数量不能匹配！")
                        continue
            else:
                image_compose()              
        else:
            continue

In [ ]:
###图片拼接结束，关闭浏览器对象###
driver.close()